In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import expipe
import pathlib
import numpy as np
import sys
sys.path.append('../mec_theta')
import data_processing as dp
from trackunitmulticomparison import TrackMultipleSessions
# from registration import store_notebook
import re
import joblib
import multiprocessing
import shutil
import psutil
import pandas as pd
import matplotlib.pyplot as plt
import quantities as pq
import exdir
from distutils.dir_util import copy_tree
from datetime import timedelta
from tqdm import tqdm_notebook as tqdm
import networkx as nx
#from nxpd import draw
%matplotlib inline

In [3]:
project_path = dp.project_path()

project = expipe.get_project(project_path)
actions = project.actions

In [4]:
output = pathlib.Path('output/identify_neurons')
output.mkdir(parents=True, exist_ok=True)

In [5]:
identify_neurons = project.require_action('identify-neurons')

In [6]:
actions

In [7]:
actions['137-310121-8'].attributes

{'registered': '2021-01-31T18:07:31',
 'datetime': '2021-01-31T17:32:11',
 'type': 'Recording',
 'tags': ['open-ephys', 'optostim, 11hz'],
 'entities': ['137'],
 'users': ['maria'],
 'location': 'ibv',
 'data': {'main': 'actions\\137-310121-8\\data\\main.exdir'}}

In [8]:
data_loader = dp.Data()

In [9]:
skip_actions = [
]

In [10]:
sessions = []
for action in actions.values():
    if action.id in skip_actions:
        continue
    if action.type != 'Recording':
        continue
    action_data_path = dp.get_data_path(action)
    exdir_object =  exdir.File(action_data_path)
    if 'processing' not in exdir_object:
        continue
        
    processing = exdir_object['processing']
    
    if not 'electrophysiology' in processing:
        continue
    elphys = processing['electrophysiology']
    if 'spikesorting' not in elphys:
        continue  

    sessions.append({
       'action': action.id,
       'session': int(action.id.split('-')[-1]),
       'entity': int(action.entities[0]),

   })
sessions = pd.DataFrame(sessions)

In [11]:
sessions

,action,session,entity
0,137-130121-1,1,137
1,137-140121-1,1,137
2,137-190121-2,2,137
3,137-310121-8,8,137


In [12]:
sessions['date'] = sessions.apply(lambda x: x.action.split('-')[1], axis=1)
sessions['entity_date'] = sessions.apply(lambda x: '-'.join(x.action.split('-')[:2]), axis=1)

In [13]:
sessions.to_csv(output / 'sessions.csv', index=False)

# Identify unique neurons

In [27]:
include = ["137-190121-2", "137-310121-8"] #list recordings to include (separated by comma)

In [28]:
sessions_include = sessions.query("action.isin(@include)")

In [29]:
sessions_include

,action,session,entity,date,entity_date
2,137-190121-2,2,137,190121,137-190121
3,137-310121-8,8,137,310121,137-310121


In [31]:
# save graphs
for entity, values in sessions_include.groupby('entity'):
    data_path = output / f'{entity}-graphs'
    if data_path.exists():
        shutil.rmtree(data_path)
#         continue
    print('Processing', entity)
    unit_matching = TrackMultipleSessions(
        actions, action_list=values.action.values.tolist(), 
        progress_bar=tqdm, verbose=False, data_path=data_path
    )
    unit_matching.do_matching()
    unit_matching.make_graphs_from_matches()
#     unit_matching.compute_time_delta_edges()
    unit_matching.compute_depth_delta_edges()
    # save graph with all dissimilarities for later use
    unit_matching.save_graphs()

Processing 137


  0%|          | 0/1 [00:00<?, ?it/s]

# Plot comparisons

In [32]:
entity = 137
unit_comp = TrackMultipleSessions(
    actions, sessions_include.action.values, data_path=f'output/identify_neurons/{entity}-graphs')

In [33]:
unit_comp.load_graphs()

In [34]:
max_dissimilarity = .05
max_depth_delta = 100

unit_comp.remove_edges_above_threshold('weight', max_dissimilarity)
unit_comp.remove_edges_above_threshold('depth_delta', max_depth_delta)

In [35]:
unit_comp.remove_edges_with_duplicate_actions()
unit_comp.identify_units()

In [39]:
unit_comp.plot_matches('template', chan_group=0, step_color=False)
plt.tight_layout()

ValueError: Number of rows must be a positive integer, not 0

<Figure size 720x0 with 0 Axes>

# Store uniqe unit ids to csv

In [22]:
max_dissimilarity = .05
max_depth_delta = 100
id_nums = {}
num = 0
for entity, values in sessions_include.groupby('entity'):
    data_path = output / f'{entity}-graphs'
    unit_matching = TrackMultipleSessions(
        actions, values.action.values.tolist(), 
        progress_bar=tqdm, verbose=False, data_path=data_path
    )
    unit_matching.load_graphs()
    # cutoff large dissimilarities
    unit_matching.remove_edges_above_threshold('weight', max_dissimilarity)
    unit_matching.remove_edges_above_threshold('depth_delta', max_depth_delta)
    unit_matching.remove_edges_with_duplicate_actions()
    unit_matching.identify_units()
    units = []
    for ch, group in unit_matching.identified_units.items():
        for unit_id, val in group.items():
            for action_id, orig_unit_ids in val['original_unit_ids'].items():
                if unit_id not in id_nums:
                    id_nums[unit_id] = num
                    num += 1
                units.extend([
                    {
                        'unit_name': name, 
                        'unit_idnum': id_nums[unit_id],
                        'unit_id': unit_id, 
                        'action': action_id,
                        'channel_group': ch,
                        'max_dissimilarity': max_dissimilarity,
                        'max_depth_delta': max_depth_delta
                    } 
                    for name in orig_unit_ids])
    units = pd.DataFrame(units)
    units = units[~units.action.isin(skip_actions)]
    units.to_csv(output / f'{entity}-units.csv', index=False)

In [23]:
unique_units = pd.concat([
    pd.read_csv(p) 
    for p in output.iterdir() 
    if p.name.endswith('-units.csv')])

In [24]:
unique_units.to_csv(output / 'units.csv', index=False)

# Store results in Expipe action

In [93]:
identify_neurons.data['sessions'] = 'sessions.csv'
identify_neurons.data['units'] = 'units.csv'

In [94]:
copy_tree(output, str(identify_neurons.data_path()))

['D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-0.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-1.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-2.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-3.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-4.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-5.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-6.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-graphs\\graph-group-7.yaml',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\133-units.csv',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\sessions.csv',
 'D:\\projects\\CA2\\actions\\identify-neurons\\data\\units.csv']

In [97]:
store_notebook(
    identify_neurons, "00-identify-neurons.ipynb")